In [1]:
!pip install pandas numpy scikit-learn nltk transformers flask

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.5 MB 311.0 kB/s eta 0:00:32
   -- ------------------------------------- 0.5/10.5 MB 311.0 kB/s eta 0:00:32
   -- ------------------------------------- 0.5/10.5 MB 311.0 kB/s eta 0:00:32
   --- ------------------------------------ 0.8/10.5 MB 369.0 kB/s eta 0:00:27
   --- ------------------------------------ 0.8/10.5 MB 369.0 kB/s eta 0:00:27
   ---- ----------------------------------- 1

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\USER\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^

Importing the dataset

In [10]:
import pandas as pd
faq_data = pd. read_csv("C:/Users/USER/Downloads/health_faq.csv" ,
                       quotechar='"',
                       encoding='utf-8',
                       engine='python'
)

faq_data.head()
                                                                

,FAQ,Answer
0,I can't stop overthinking. How do I break the ...,Overthinking often stems from anxiety. Try the...
1,Why do I feel tired all the time even after sl...,"Chronic fatigue can link to depression, stress..."
2,How do I set boundaries without feeling guilty?,Boundaries protect your energy. Start with: 'I...
3,I had a bad therapy session. Should I quit?,Not all therapists are the right fit. It's oka...
4,Can exercise really help my mental health?,Yes! Even 10-minute walks release mood-boostin...


Basic Keyword Checking

In [20]:
risk_keywords = ["suicide", "overthinking", "end it all", "can't cope","guilty","tired"]

def check_risk_words(text):
    text = text.lower()
    found_words = [word for word in risk_keywords if word in text]
    return found_words

In [22]:
print(check_risk_words("i had a bad therapy, i feel guilty"))

['guilty']


Create a Sentiment analysis

In [27]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')  # Download once




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...


True

Run and observe the Sentiment

In [28]:
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score < -0.5:
        return "High Risk"
    else:
        return "Low/Moderate Risk"



In [29]:
print(analyze_sentiment("I am feeling really sad and hopeless."))
print(analyze_sentiment("I am doing okay today."))

High Risk
Low/Moderate Risk


Generate Chatbot Response

In [40]:
def generate_response(user_input):
    risk_words = check_risk_words(user_input)
    sentiment = analyze_sentiment(user_input)

    if risk_words:
        return "⚠ I'm concerned about your safety. Please call Nigeria's mental health helpline at 0800-XXX-XXXX.", "Critical"
    elif sentiment == "High Risk":
        return "It sounds like you're going through a tough time. Would you like to talk more about what's bothering you?", "High"
    else:
        # Return random FAQ answer for now
        return faq_da.sample()['Answer'].values[0], "Low"


In [34]:
print(generate_response("I feel like I want to end it all."))
print(generate_response("I'm very anxious about work."))
print(generate_response("i am overthinking"))

("⚠ I'm concerned about your safety. Please call Nigeria's mental health helpline at 0800-XXX-XXXX.", 'Critical')
("Holidays highlight 'shoulds' (family, joy) vs reality. Scale back obligations. Create new traditions that honor YOUR needs.", 'Low')
("⚠ I'm concerned about your safety. Please call Nigeria's mental health helpline at 0800-XXX-XXXX.", 'Critical')


Make a Flask Chat API

In [38]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get('message', '')
    response, risk = generate_response(user_message)
    save_chat(user_message, response, risk)
    return jsonify({'reply': response, 'risk': risk})

if __name__ == "_main_":
    app.run(port=5000)

In [ ]:
import random
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Predefined answers
answers = [
    "You're doing your best, and that's enough.",
    "Try to breathe deeply. You're not alone.",
    "Talk to someone you trust. You matter.",
    "It's okay to feel overwhelmed sometimes.",
    "You're doing your best, and that's enough.",
    "It's okay not to be okay. Take one breath at a time.",
    "You’re not alone. Many people feel this way too — and they got through it.",
    "Be kind to yourself. You deserve rest and peace.",
    "Even the strongest people need support. Asking for help is strength.",
    "Talk to someone you trust. You matter and your feelings are valid.",
    "Try grounding yourself: What can you see, hear, smell, feel right now?",
    "Healing takes time — but every small step counts.",
    "It’s brave to open up about how you feel. You’ve already done something powerful.",
    "I’m proud of you for sharing. That’s how healing starts.",
    "Would you like to talk more about what's bothering you?"
]

# Keywords that raise concern
risk_keywords = ["suicide", "hopeless", "end it all", "kill myself", "can't cope"]

# Sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Check sentiment
def analyze_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return "High Risk" if score < -0.5 else "Moderate/Low Risk"

# Check risky words
def check_risk_words(text):
    return [word for word in risk_keywords if word in text.lower()]

# Chatbot logic
def generate_response(user_input):
    risk_words = check_risk_words(user_input)
    sentiment = analyze_sentiment(user_input)

    if risk_words:
        return ("⚠ I'm really concerned about your safety. Please speak to someone you trust or call a mental health helpline.", "Critical")
    elif sentiment == "High Risk":
        return ("You sound overwhelmed. I'm here for you. Want to talk more?", "High")
    else:
        return (random.choice(answers), "Moderate/Low")

# Simple chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Take care of yourself. 💚")
        break
    response, risk = generate_response(user_input)
    print(f"Bot ({risk}): {response}")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


You:  i feel realy concerned about my safety


Bot (Moderate/Low): Try to breathe deeply. You're not alone.


You:  quity


Bot (Moderate/Low): You're doing your best, and that's enough.


...

Bot (High): You sound overwhelmed. I'm here for you. Want to talk more?


You:  i am overthink


Bot (Moderate/Low): It's okay to feel overwhelmed sometimes.


You:  i am lonely


Bot (Moderate/Low): Try to breathe deeply. You're not alone.
